$\beta = -1$ means that $\beta$ is selected at random in each iteration.

In [32]:
from grasp_betas import read_results

save = True
names = [
    "eil101_68_33", # 0
    "ch150_100_50_0",
    "pr439_293_146_0", # 2
    "rat783_522_261_0",
    "rl1323_882_441_0", # 4
    "anpcp_882_441_0",
]
name = names[0]

betas = read_results(name)

In [33]:
betas

,i,p,alpha,0.0,0.25,0.5,0.75,1.0,-1,d_0.0,d_0.25,d_0.5,d_0.75,d_1.0,d_-1
0,0,3,2,46,46,46,46,46,46,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
1,0,6,2,32,31,31,31,31,31,3.225806,0.000000,0.000000,0.000000,0.0,0.000000
2,1,3,2,49,49,49,49,49,49,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
3,1,6,2,32,32,32,32,31,32,3.225806,3.225806,3.225806,3.225806,0.0,3.225806
4,2,3,2,50,50,50,50,50,50,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
5,2,6,2,34,34,34,34,34,34,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
6,3,3,2,48,48,48,48,48,48,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
7,3,6,2,33,31,31,31,31,31,6.451613,0.000000,0.000000,0.000000,0.0,0.000000
8,4,3,2,44,44,44,44,44,44,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
9,4,6,2,30,30,30,30,30,30,0.000000,0.000000,0.000000,0.000000,0.0,0.000000


In [34]:
diffs = betas.iloc[:, 10:17]
diffs

,d_0.25,d_0.5,d_0.75,d_1.0,d_-1
0,0.000000,0.000000,0.000000,0.0,0.000000
1,0.000000,0.000000,0.000000,0.0,0.000000
2,0.000000,0.000000,0.000000,0.0,0.000000
3,3.225806,3.225806,3.225806,0.0,3.225806
4,0.000000,0.000000,0.000000,0.0,0.000000
5,0.000000,0.000000,0.000000,0.0,0.000000
6,0.000000,0.000000,0.000000,0.0,0.000000
7,0.000000,0.000000,0.000000,0.0,0.000000
8,0.000000,0.000000,0.000000,0.0,0.000000
9,0.000000,0.000000,0.000000,0.0,0.000000


In [35]:
avgs = diffs.mean()

In [36]:
bests = (diffs == 0).sum()

In [37]:
import pandas as pd

data = {
    "average": avgs,
    "#bests": bests
}
pd.DataFrame(data.values(), index=data.keys())

,d_0.25,d_0.5,d_0.75,d_1.0,d_-1
average,0.322581,0.322581,0.322581,0.0,0.322581
#bests,9.000000,9.000000,9.000000,10.0,9.000000
